# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [5]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import os

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [6]:
import re
import time 
import json
import requests
import collections
import torch 
import torch.nn as nn
import torch.nn.functional as F
from urllib.parse import parse_qs

/home/harsh1/github-repos/Trexquant/venv/lib/python3.8/site-packages/torch/_subclasses/functional_tensor.py:258: UserWarning: Failed to initialize NumPy: numpy.core.multiarray failed to import (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [ ]:
from hangman_env import HangmanEnv

class RLPolicyNet(nn.Module):
    def __init__(self , vocab_size , embedding_dim = 32 , hidden_dim = 64):
        super(RLPolicyNet , self).__init__()

        self.embedding = nn.Embedding(vocab_size +1, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim , hidden_dim , batch_first=True)
        self.fc = nn.Linear(hidden_dim , vocab_size)


    def forward(self , x , hidden = None):
        emb , hidden = self.embedding(x) , hidden
        out , hidden = self.lstm(emb , hidden)
        logits = self.fc(out[: , -1 , :])
        return logits , hidden

In [23]:


import os
import torch
from vocab import VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, LETTERS, LETTER2IDX

from train_hangman_rl import ActorCriticNet

class RLAgent:
    def __init__(self, policy_path, device="cpu"):
        self.device = device
        self.letters = LETTERS
        self.vocab = LETTER2IDX
        # initialize net with the same vocab size & dims used in training
        self.net = ActorCriticNet(
            vocab_size=VOCAB_SIZE,
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM
        ).to(self.device)

        if os.path.isfile(policy_path):
            checkpoint = torch.load(policy_path, map_location=self.device)
            state = checkpoint.get("model_state", checkpoint)
            # non‐strict loading skips any leftover mismatches
            self.net.load_state_dict(state, strict=False)
            print(f"[RLAgent] Loaded policy (strict=False) from {policy_path}")
        else:
            raise FileNotFoundError(f"Policy file not found: {policy_path}")

        self.net.eval()
    
    def encode_state(self , pattern , guessed_letters):

        seq = []
        for ch in pattern:
            if ch in self.vocab:
                seq.append(self.vocab[ch])
            else:
                seq.append(len(self.vocab)-1)

        return torch.tensor([seq] , dtype = torch.long , device = self.device)
    
    def select_letter(self , pattern , guessed_letters):

        state = self.encode_state(pattern , guessed_letters)
        with torch.no_grad():
            logits, _ , _  = self.net(state)
            probs = F.softmax(logits , dim =-1).squeeze(0)

        mask = torch.ones_like(probs)
        for ch in guessed_letters:
            if ch in self.vocab:
                mask[self.vocab[ch]] = 0.0
        probs = probs * mask

        if probs.sum() == 0:
            probs = torch.ones_like(probs)
        probs = probs / probs.sum()

        choice = torch.multinomial(probs, num_samples=1).item()
        return self.letters[choice]
    

        

In [24]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None , rl_policy_path="models/policy.pt"):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []

        self.agent = RLAgent(rl_policy_path, device="cpu")
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word):
        # clean and filter by pattern
        clean = word[::2].replace("_", ".")
        L = len(clean)
        self.current_dictionary = [
            w for w in self.current_dictionary
            if len(w) == L and re.fullmatch(clean, w)
        ]

        # RL pick
        letter = self.agent.select_letter(clean, self.guessed_letters)

        # fallback to frequency if needed
        if letter not in self.agent.letters:
            c = collections.Counter("".join(self.current_dictionary))
            for l, _ in c.most_common():
                if l not in self.guessed_letters:
                    letter = l
                    break
            else:
                for l, _ in self.full_dictionary_common:
                    if l not in self.guessed_letters:
                        letter = l
                        break
        return letter


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [25]:
api = HangmanAPI(access_token="8e320fc4b246a146384b9feb3ded7e", timeout=2000)


[RLAgent] Loaded policy (strict=False) from models/policy.pt


/tmp/ipykernel_3814/1665768981.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(policy_path, map_location=self.device)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [27]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 3867cf843765. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '3867cf843765', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ e '}
Guessing letter: a
Sever response: {'game_id': '3867cf843765', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ a _ _ _ _ e '}
Guessing letter: i
Sever response: {'game_id': '3867cf843765', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ a _ _ i _ e '}
Guessing letter: r
Sever response: {'game_id': '3867cf843765', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ a _ _ i _ e '}
Guessing letter: s
Sever response: {'game_id': '3867cf843765', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ _ a _ _ i _ e '}
Guessing letter: t
Sever response: {'game_id': '3867cf843765', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ _ t a _ t i _ e '}
Guessing letter: n
Sever response: {'game_id': '3867cf843765', 'status': 'ongoing', 'tries_

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [30]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game


IndexError: list index out of range

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [32]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.000
